The code consists of the following parts:

1. **Section 1** - Prepare datasets
2. **Section 2** - Modelling
3. **Section 3** - Fine-tuning
4. **Section 4** - Evaluation

In [57]:
# Load Libraries & Dataset
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from utils import merge_data
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score, roc_curve, auc

data_path = '/Volumes/WORK/dataset/retail_project/cleaned_data/cleaned_data.csv'
products_data_path = '/Volumes/WORK/dataset/retail_project/cleaned_data/potential_products.csv'

In [59]:
potential_products_df = pd.read_csv(products_data_path)
data = pd.read_csv(data_path)
data['invoice_date'] = pd.to_datetime(data['invoice_date'])
data['potential_products'] = np.where(data['stock_code'].isin(potential_products_df['stock_code'].unique()),1,0)

In [60]:
df_2010 = data[data['invoice_date'].dt.year == 2010].reset_index(drop=True)

In [61]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:  # 9, 10, 11
        return 'autumn'


In [63]:
df_2010['season'] = df_2010['month'].apply(get_season)

In [67]:
df_2010['num_invoice_winter'] = (df_2010[df_2010['season'] == 'winter'].groupby('customer_id')['invoice'].transform('nunique'))
df_2010['num_invoice_winter'] = df_2010['num_invoice_winter'].fillna(0) 

df_2010['num_invoice_spring'] = (df_2010[df_2010['season'] == 'spring'].groupby('customer_id')['invoice'].transform('nunique'))
df_2010['num_invoice_spring'] = df_2010['num_invoice_spring'].fillna(0) 

df_2010['num_invoice_summer'] = (df_2010[df_2010['season'] == 'summer'].groupby('customer_id')['invoice'].transform('nunique'))
df_2010['num_invoice_summer'] = df_2010['num_invoice_summer'].fillna(0) 

df_2010['num_invoice_autumn'] = (df_2010[df_2010['season'] == 'autumn'].groupby('customer_id')['invoice'].transform('nunique'))
df_2010['num_invoice_autumn'] = df_2010['num_invoice_autumn'].fillna(0) 

In [69]:
df_2010 = df_2010[['customer_id','num_invoice_winter','num_invoice_spring','num_invoice_summer','num_invoice_autumn']].drop_duplicates()

In [71]:
data = merge_data(data, df_2010, 'customer_id', 'customer_id', None, 'left')

## Section 1: Prepare datasets

In [75]:
def grouped_data(df, group_col, value_col, group_type):
    grouped_df = df.groupby(group_col).agg({value_col: group_type}).reset_index()
    return grouped_df

def categorize(row, ox, oy):
    if row[ox] == 5 and 4 <= row[oy] <= 5:
        return 'Champions'
    elif 3 <= row[ox] <= 4 and 4 <= row[oy] <= 5:
        return 'Loyal Customer'
    elif 4 <= row[ox] <= 5 and 2 <= row[oy] <= 3:
        return 'Potential Loyalist'
    elif row[ox] == 5 and row[oy] == 1:
        return 'New Customers'
    elif row[ox] == 4 and row[oy] == 1:
        return 'Promising'
    elif row[ox] == 3 and 1 <= row[oy] <= 2:
        return 'Needs Attention'
    elif row[ox] == 3 and row[oy] == 3:
        return 'About to Sleep'
    elif 1 <= row[ox] <= 2 and row[oy] == 5:
        return 'Can\'t Loose Them'
    elif 1 <= row[ox] <= 2 and 3 <= row[oy] <= 4:
        return 'At Risk'
    elif 1 <= row[ox] <= 2 and 1 <= row[oy] <= 2:
        return 'Hibernating'
    else:
        return 'Unclassified'


def segment_customers_rfm(df):
    # df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    reference_date = df['invoice_date'].max() + pd.Timedelta(days=1)

    # Calculate R,F,M
    rfm = df.groupby('customer_id').agg({
        'invoice_date': lambda x: (reference_date - x.max()).days,  # Recency
        'invoice': 'nunique',                                       # Frequency
        'revenue': 'sum'                                            # Monetary
    }).reset_index()
    
    rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']
    rfm['recency_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
    rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])

    rfm['segment_customers'] = rfm.apply(categorize, axis=1, args=('recency_score', 'monetary_score'))
    return rfm
    


def created_feature_engineering(data):
    df = data.copy()
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    ####### Feature engineering
    df_feat_eng = df[['customer_id','label']].drop_duplicates()
    
    ####  1. Number of Invoices per Customer (num_invoices)
    # Compute groupby aggregations
    invoices_df = grouped_data(df[df['invoice_type']=='purchase'], 'customer_id', 'invoice', 'nunique')
    invoices_df = invoices_df.rename(columns={'invoice': 'num_invoice'})
    
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, invoices_df[['customer_id','num_invoice']], 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['num_invoice'] = df_feat_eng['num_invoice'].fillna(0)
    
    ####  2. Average Revenue per Invoice per Customer (avg_invoice_revenue)
    # Compute groupby aggregations
    revenue_per_invoice = grouped_data(df, ['customer_id', 'invoice'], 'revenue', 'sum')
    avg_invoice_revenue =  revenue_per_invoice.groupby('customer_id').agg(avg_invoice_revenue = ('revenue', 'mean')).reset_index()
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, avg_invoice_revenue[['customer_id','avg_invoice_revenue']], 'customer_id', 'customer_id', None, 'left')
    
    ####  3. Average Purchase Interval per Customer (avg_purchase_interval)
    # Compute groupby aggregations
    invoice_dates = df[df['invoice_type']=='purchase'][['customer_id', 'invoice_date']].drop_duplicates(keep='first')
    invoice_dates['invoice_date'] = pd.to_datetime(invoice_dates['invoice_date'])
    invoice_dates = invoice_dates.sort_values(by=['customer_id', 'invoice_date'])
    invoice_dates['purchase_interval'] = invoice_dates.groupby('customer_id')['invoice_date'].diff().dt.days
    avg_purchase_interval = grouped_data(invoice_dates, 'customer_id', 'purchase_interval', 'mean')
    avg_purchase_interval = avg_purchase_interval.rename(columns={'purchase_interval': 'avg_purchase_interval'})
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, avg_purchase_interval[['customer_id','avg_purchase_interval']], 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['avg_purchase_interval'] = df_feat_eng['avg_purchase_interval'].fillna(1000)
    
    ####  4. Average Product Price per Customer (avg_price)
    # Compute groupby aggregations
    stock_code_price_per_customer = df[df['invoice_type']=='purchase'][['customer_id', 'invoice_date','price']].sort_values('invoice_date', ascending=False)
    stock_code_price_per_customer = stock_code_price_per_customer.drop_duplicates(subset='customer_id', keep='first').reset_index(drop=True)
    avg_price = grouped_data(stock_code_price_per_customer, 'customer_id', 'price', 'mean')
    avg_price = avg_price.rename(columns={'price': 'avg_price'})
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, avg_price[['customer_id','avg_price']], 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['avg_price'] = df_feat_eng['avg_price'].fillna(0)
    
    ####  5. Number of Product Types per Customer (num_product_types)
    # Compute groupby aggregations
    num_product_types = grouped_data(df[df['invoice_type']=='purchase'], 'customer_id', 'stock_code', 'nunique')
    num_product_types = num_product_types.rename(columns={'stock_code': 'num_product_types'})
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, num_product_types[['customer_id','num_product_types']], 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['num_product_types'] = df_feat_eng['num_product_types'].fillna(0)
    
    ####  6. Revenue per Customer (revenue)
    # Compute groupby aggregations
    revenue = grouped_data(df, 'customer_id', 'revenue', 'sum')
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, revenue[['customer_id','revenue']], 'customer_id', 'customer_id', None, 'left')

    #### 7. Top10 products purchase count (top10_products_purchase_count)
    # Compute groupby aggregations
    top10_products_purchase_count = grouped_data(df[df['potential_products']==1], 'customer_id', 'quantity', 'sum')
    top10_products_purchase_count = top10_products_purchase_count.rename(columns={'quantity': 'top10_products_purchase_count'})
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, top10_products_purchase_count[['customer_id','top10_products_purchase_count']], 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['top10_products_purchase_count'] = df_feat_eng['top10_products_purchase_count'].fillna(0)

    #### 8. Recency top10 purchase days (recency_top10_purchase_days)
    last_top10_purchase_date = df[df['potential_products']==1].groupby('customer_id').agg(last_top10_purchase_date = ('invoice_date', 'max')).reset_index()
    reference_date = df['invoice_date'].max() + pd.Timedelta(days=1)
    last_top10_purchase_date['recency_top10_purchase_days'] = (
        reference_date - last_top10_purchase_date['last_top10_purchase_date']
    ).dt.days
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, last_top10_purchase_date[['customer_id','recency_top10_purchase_days']], 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['recency_top10_purchase_days'] = df_feat_eng['recency_top10_purchase_days'].fillna(1000)


    #### 9. Revenue top10
    # revenue_top10 = df[df['invoice_type']=='return'].groupby('customer_id').agg(revenue_top10 = ('revenue', 'sum')).reset_index()
    # Merge with feature engineering data
    
    df_2010 = df[['customer_id','num_invoice_autumn']].drop_duplicates()
    
    df_feat_eng = merge_data(df_feat_eng, df_2010, 'customer_id', 'customer_id', None, 'left')
    df_feat_eng['num_invoice_autumn'] = df_feat_eng['num_invoice_autumn'].fillna(0)

    
    

    #### 10. Revenue trend (revenue_trend)
    df_sorted = data.sort_values(by=['customer_id', 'invoice_date'])
    first_revenue = data.groupby('customer_id').first()['revenue']
    last_revenue = data.groupby('customer_id').last()['revenue']
    revenue_trend = last_revenue - first_revenue
    trend_df = revenue_trend.reset_index().rename(columns={'revenue': 'revenue_trend'})
    # Merge with feature engineering data
    df_feat_eng = merge_data(df_feat_eng, trend_df[['customer_id','revenue_trend']], 'customer_id', 'customer_id', None, 'left')

    return df_feat_eng


def created_dataset(min_time, max_time, data):
    #### Get months
    months = data[(data['month']<=max_time)&(data['month']>=min_time)]['month'].unique()

    #### Split dataset
    # Feature data
    feature_df =  data[data['month'].isin(sorted(months, reverse=False)[:5])].copy()
    feature_customers = feature_df['customer_id'].unique()
    
    # Labled data
    labeled_data = data[
        (data['month'].isin(sorted(months, reverse=False)[5:7])) &
        (data['potential_products']==1) &
        (data['customer_id'].isin(feature_customers))
    ]
    
    # Assign label 
    return_customers =  labeled_data[labeled_data['invoice_type']=='purchase']['customer_id'].unique()  
    feature_df['label'] = np.where(feature_df['customer_id'].isin(return_customers),1,0)

    #### Created feature engineering
    x_data =  created_feature_engineering(feature_df)

    scaler = StandardScaler()
    cols_to_scale = ['num_invoice', 'avg_invoice_revenue',
       'avg_purchase_interval', 'avg_price', 'num_product_types', 'revenue',
       'top10_products_purchase_count', 'recency_top10_purchase_days',
       'num_invoice_autumn', 'revenue_trend']
    x_data[cols_to_scale] = scaler.fit_transform(x_data[cols_to_scale])



    

    #### Segment customers rfm
    segment_customers = segment_customers_rfm(feature_df)
    x_data = merge_data(x_data, segment_customers[['customer_id','segment_customers']], 'customer_id', 'customer_id', None, 'left')

    return x_data

In [77]:
test_dataset = created_dataset('2011-05', '2011-11', data)
validation_dataset = created_dataset('2011-03', '2011-09', data)
train_dataset = created_dataset('2011-01', '2011-07', data)

In [78]:
test_dataset

,customer_id,label,num_invoice,avg_invoice_revenue,avg_purchase_interval,avg_price,num_product_types,revenue,top10_products_purchase_count,recency_top10_purchase_days,num_invoice_autumn,revenue_trend,segment_customers
0,15606,0,0.101135,-0.087617,-0.932569,-0.057982,-0.399224,-0.055783,-0.198521,1.093569,0.249590,0.052540,Loyal Customer
1,14800,0,0.890614,-0.388136,-0.978077,-0.078090,1.471063,0.051072,-0.198521,1.093569,0.249590,0.020631,Champions
2,16931,1,1.680093,-0.335952,-1.001718,-0.077626,3.305725,0.128418,0.034724,-0.904029,-0.129212,0.020631,Champions
3,15708,1,-0.162025,-0.196936,-0.863681,0.047352,0.313266,-0.030661,0.027656,-0.889121,-0.129212,0.013446,At Risk
4,14304,0,-0.425185,0.081330,1.048492,-0.033853,-0.434848,-0.166795,-0.198521,1.093569,-0.318612,-0.004455,At Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,15034,1,-0.425185,-0.098186,1.048492,-0.058601,0.972320,-0.120384,-0.170249,-1.033936,0.154889,0.027989,Potential Loyalist
2762,15185,0,-0.425185,-0.089217,1.048492,-0.078090,-0.363599,-0.181483,-0.198521,1.093569,-0.508013,0.008228,Potential Loyalist
2763,16404,0,-0.425185,0.220895,1.048492,-0.007558,-0.327975,-0.154775,-0.141977,-1.033936,-0.508013,0.016818,Potential Loyalist
2764,12842,0,-0.425185,1.076110,1.048492,-0.039421,-0.381411,-0.081123,0.253834,-1.033936,-0.413313,0.090270,Champions


## Section 2: Modelling

In [131]:
def prepare_data(df, label_col):
    x = df.drop(columns=[label_col, 'segment_customers'])
    y = df[label_col]
    return x,y


def train_model(x_train, y_train):
    model = LogisticRegression(
        max_iter=1000,
        verbose=0,
        random_state=42
    )
    model.fit(x_train, y_train)
    return model

def predict(model, x_val, threshold=0.5):
    y_pred_prob = model.predict(x_val)
    y_pred = (y_pred_prob >= threshold).astype(int)
    return y_pred_prob, y_pred

def evaluate(y_true, y_pred, y_pred_prob):
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_prob)
    recall   = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)  
    return accuracy, auc, recall, precision, f1

In [160]:
x_train, y_train = prepare_data(train_dataset, 'label')
x_val, y_val  = prepare_data(validation_dataset, 'label')
x_test, y_test  = prepare_data(test_dataset, 'label')


#Train model
model = train_model(x_train, y_train)

# Predict test
y_pred_prob, y_pred = predict(model, x_test, threshold=0.5)

# Evaluation
accuracy, score_auc, recall, precision, f1 = evaluate(y_test, y_pred, y_pred_prob)

print(f"Validation Precision: {precision:.4f}")

Validation Precision: 0.6944


/opt/anaconda3/envs/data_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Section 3: Fine-tuning

## Section 4: Evaluation

### 4.2: Segment customers rfm 

In [146]:
y_pred_prob, y_pred = predict(model, x_val, threshold=0.5)
validation_dataset['pred_prob'] = y_pred_prob

# Evaluation
accuracy, auc, recall, precision, f1 = evaluate(y_val, y_pred, y_pred_prob)

In [148]:
rfm_top_customers =  validation_dataset[validation_dataset['segment_customers'].isin(["Champions","Loyal Customer","Can\'t Loose Them"])]

In [150]:
rfm_top_customers['customer_id'].nunique()

906

In [152]:
rfm_top_customers[rfm_top_customers['label']==1]['customer_id'].nunique()/rfm_top_customers['customer_id'].nunique()

0.4116997792494481

In [154]:
prob_top_customers = validation_dataset.sort_values(by='pred_prob',ascending=False).head(rfm_top_customers['customer_id'].nunique())

In [156]:
prob_top_customers[prob_top_customers['label']==1]['customer_id'].nunique()/prob_top_customers['customer_id'].nunique()

0.3509933774834437

In [64]:
prob_top_customers['customer_id'].nunique()

906

In [360]:
test_dataset[test_dataset['label']==1]['customer_id'].nunique()

715

In [66]:
prob_top_customers

,customer_id,label,num_invoice,avg_invoice_revenue,avg_purchase_interval,avg_price,num_product_types,revenue,top10_products_purchase_count,recency_top10_purchase_days,num_invoice_autumn,revenue_trend,segment_customers,pred_prob
254,15498,1,2.348678,-0.014982,-1.055046,-0.169644,2.057672,0.946119,0.820279,-0.898014,1.418034,-0.002119,Loyal Customer,0.881297
109,18102,1,4.579363,9.004454,-1.058505,0.061829,0.184068,24.377721,4.367497,-0.874271,7.318824,-0.497422,Champions,0.875709
147,17841,1,12.107923,-0.210456,-1.071042,-0.169644,15.543723,3.371660,1.102530,-0.984352,8.425222,0.013498,Champions,0.875581
139,17675,1,2.627514,0.192886,-1.053268,0.303082,1.921055,1.753209,2.994380,-0.960609,3.630831,0.023138,Champions,0.874957
968,17677,1,2.348678,0.056696,-1.054214,-0.197030,0.867153,1.458020,4.062360,-0.960609,3.630831,-0.010859,Champions,0.866762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,16947,0,-0.439677,0.191889,1.003322,-0.140955,-0.362401,-0.187726,-0.056991,-0.759873,-0.333763,0.020516,At Risk,0.231142
1311,15147,0,-0.439677,-0.046133,1.003322,-0.169644,-0.225784,-0.131003,-0.148532,-0.749080,-0.518162,-0.021656,At Risk,0.231095
1458,17463,0,-0.160842,-0.342622,-0.958515,0.025967,-0.147717,-0.152725,-0.148532,-0.887222,-0.333763,-0.008160,About to Sleep,0.230770
1682,15518,1,0.117994,-0.167794,-0.993882,-0.124002,0.593919,-0.089569,-0.171417,-0.923915,0.127236,0.012740,Champions,0.230065


In [68]:
validation_dataset['customer_id'].nunique()

2621

In [70]:
906/2621

0.3456695917588707

In [82]:
prob_top_customers = validation_dataset.sort_values(by='pred_prob',ascending=False).head(635)

In [84]:
prob_top_customers[prob_top_customers['label']==1]['customer_id'].nunique()

349

In [72]:
validation_dataset[validation_dataset['label']==1]['customer_id'].nunique()

635

In [86]:
349/635

0.5496062992125984